## Module 2 Homework

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [Colab](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/Module2_Colab_Working_with_the_data.ipynb) covered at the livestream to re-use the code snippets.

---
### Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happenned of Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).
You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`). You should see 32 records in total, 24 of it is not null.

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)

---

In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

response = requests.get("https://stockanalysis.com/ipos/filings/")
soup = BeautifulSoup(response.text, 'html.parser')

# 假設數據在一個表格內，找到該表格並提取數據
table = soup.find('table')
rows = table.find_all('tr')
data = []
for row in rows:
    cols = row.find_all('td')
    data.append([ele.text.strip() for ele in cols if ele])

df = pd.DataFrame(data, columns=['filing Date', 'Symbol', 'Company Name', 'Price Range', 'Shares Offered' ])
df = df.iloc[1:]  # 根據表格的實際布局調整索引

In [44]:
# replace the 'Shares Offeered' column comma to empty
df['Shares Offered'] = df['Shares Offered'].str.replace(',', '')

In [45]:
df.head()

,filing Date,Symbol,Company Name,Price Range,Shares Offered
1,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
2,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
3,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
4,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
5,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-


In [46]:
# replace df['Shares Offered'] non-numeric value to np.nan
df['Shares Offered'] = pd.to_numeric(df['Shares Offered'], errors='coerce')

In [47]:
# df['filing Date'] = pd.to_datetime(df['filing Date'])
df['filing Date'] = pd.to_datetime(df['filing Date'], format='%b %d, %Y')


In [48]:
df.to_csv('ipo_filings.csv', index=False)

In [49]:
def extract_avg_price(price_str):
    # Remove the dollar sign and split by '-'
    prices = price_str.replace('$', '').split('-')
    # Filter out empty strings and convert the remaining to float
    prices = [float(price) for price in prices if price.strip()]
    # Calculate the average if it's a range, return the single value, or None if no valid prices
    if len(prices) == 2:
        return sum(prices) / 2
    elif len(prices) == 1:
        return prices[0]
    else:
        return None

# Apply the function to the 'Price Range' column
df['Avg_price'] = df['Price Range'].apply(extract_avg_price)

In [50]:
df.head()

,filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price
1,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,NaN
2,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.0
3,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.0
4,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.0
5,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN


In [51]:
df.to_csv('./ipo_filings.csv', index=False)

In [52]:
# if df['Avg_price'] and df['Shares Offered'] both are not null or np.nan, then calculate the 'Shares_offered_value'
import numpy as np

# Calculate 'Shares_offered_value' only where both 'Avg_price' and 'Shares Offered' are not NaN
df['Shares_offered_value'] = np.where(df['Avg_price'].notna() & df['Shares Offered'].notna(),
                                      df['Avg_price'] * df['Shares Offered'],
                                      np.nan)

In [53]:
df.to_csv('./ipo_filings.csv', index=False)

In [54]:
# Filter for entries from 2023 and on Fridays
friday_filings_2023 = df[(df['filing Date'].dt.year == 2023) & (df['filing Date'].dt.dayofweek == 4)]

# Calculate the sum of 'Shares_offered_value', convert to millions and round to the nearest integer
total_sum_millions = int(round(friday_filings_2023['Shares_offered_value'].sum() / 1e6))

total_sum_millions

286

### Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 185 tickers. Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day).
Find X, when the 75% quantile growth (among 185 investments) is the highest. 

HINTs:
* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
* You can use the `DataFrame.describe()` function to get mean, min, max, 25-50-75% quantiles.


Addtional: 
* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?


---

In [55]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def clawer(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 假設數據在一個表格內，找到該表格並提取數據
    table = soup.find('table')
    rows = table.find_all('tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        data.append([ele.text.strip() for ele in cols if ele])

    df = pd.DataFrame(data, columns=['IPO Date', 'Symbol', 'Company Name', 'IPO Price', 'Current','Return'])
    df = df.iloc[1:]  # 根據表格的實際布局調整索引
    return df

In [56]:
url1 = 'https://stockanalysis.com/ipos/2023/'
df1 = clawer(url1)

In [57]:
url2 = 'https://stockanalysis.com/ipos/2024/'
df2 = clawer(url2)

In [58]:
# concate df1 and df2
df = pd.concat([df1, df2])
df['IPO Date'] = pd.to_datetime(df['IPO Date'])
filtered_symbol_data = df[df['IPO Date'] < '2024-03-01']
# df.to_csv('symbols.csv', index=False)



In [59]:
filtered_symbol_data.sort_values(by='IPO Date', ascending=True)
filtered_symbol_data.to_csv('symbols.csv', index=False)


In [60]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
# 假設已經有了從網站上抓取的 IPO 數據和股票代碼的 DataFrame
# 需要有以下列：'Ticker', 'IPO Date'
symbol_data = pd.read_csv('symbols.csv')


In [61]:
# 過濾 IPO 日期在 2024-03-01 之前的股票
symbol_data['IPO Date'] = pd.to_datetime(symbol_data['IPO Date'])

In [62]:
filtered_symbol_data.__len__()

185

In [63]:
#delete the row['Symbol'] == 'RYZB'
filtered_symbol_data = filtered_symbol_data[filtered_symbol_data['Symbol'] != 'RYZB']
# modify the row['Symbol'] == 'PTHR' to 'PTHRU'
filtered_symbol_data['Symbol'] = filtered_symbol_data['Symbol'].replace('PTHR', 'HOVR')


In [64]:
# creat symbol list
symbol_list = list(filtered_symbol_data['Symbol'])

In [65]:
len(symbol_list)

184

In [66]:
import yfinance as yf
# create download_ipo_data to download stock data
def download_ipo_data(symbol_list):
    dataframe = {}
    for symbol in symbol_list:
        symbol = symbol.upper()
        df = yf.download(symbol, period='max',interval='1d')
        if not df.empty:
            dataframe[f'df_{symbol}'] = df
        else:
            print(f'no data found for {symbol}')
    return dataframe

dfs = download_ipo_data(symbol_list)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [67]:
len(dfs)

184

In [68]:
# from filtered_symbol_data get IPO Date and Symbol, if i hold from IPO Date to IPO Date + 30 days, get the df_Symbol['Adj Close']   
# calculate the 75% quantile growth
# write a for loop

In [69]:
# filtered_symbol_data = filtered_symbol_data.iloc[:1]

In [70]:
import pandas as pd
import numpy as np

def calculate_growth(filtered_symbol_data, dfs):
    results = []
    for index, row in filtered_symbol_data.iterrows():
        symbol = row['Symbol']
        ipo_date = pd.to_datetime(row['IPO Date'])  # Ensure ipo_date is a datetime object

        df_symbol = dfs.get(f'df_{symbol}', pd.DataFrame())  # Get the dataframe for the symbol
        if not df_symbol.empty:
            # Ensure the DataFrame index is a DatetimeIndex
            if not isinstance(df_symbol.index, pd.DatetimeIndex):
                df_symbol.index = pd.to_datetime(df_symbol.index)

            if ipo_date in df_symbol.index:
                ipo_price = df_symbol.at[ipo_date, 'Adj Close']  # Use 'at' for scalar value
                ipo_index = df_symbol.index.get_loc(ipo_date)
            else:
                # If IPO date is not in the index, use the first available date in the DataFrame
                ipo_date = df_symbol.index.min()
                if pd.isnull(ipo_date):
                    print(f"No initial trading data available for symbol {symbol}")
                    continue
                ipo_price = df_symbol.at[ipo_date, 'Adj Close']
                ipo_index = df_symbol.index.get_loc(ipo_date)

            # Iterate over the next 30 trading days
            for x in range(1, 31):  # Days from 1 to 30
                future_index = ipo_index + x
                if future_index < len(df_symbol.index):
                    future_date = df_symbol.index[future_index]
                    future_price = df_symbol.at[future_date, 'Adj Close']
                    if not np.isnan(ipo_price) and not np.isnan(future_price):
                        growth = (future_price - ipo_price) / ipo_price
                        results.append({'Symbol': symbol, 'Days': x, 'Growth': growth})
        else:
            print(f"No data found for symbol {symbol}")

    return results

In [71]:

results = calculate_growth(filtered_symbol_data, dfs)


In [72]:
# convert results to dataframe
results_df = pd.DataFrame(results)


In [73]:
results_df

,Symbol,Days,Growth
0,IROH,1,0.000999
1,IROH,2,0.000500
2,IROH,3,0.000999
3,IROH,4,0.000000
4,IROH,5,0.000999
...,...,...,...
5500,ROMA,26,-0.506897
5501,ROMA,27,-0.562069
5502,ROMA,28,-0.517241
5503,ROMA,29,-0.472414


In [74]:
# Calculate the 75% quantile growth for each day
quantile_75_growth = results_df.groupby('Days')['Growth'].quantile(0.75).reset_index()

In [75]:
quantile_75_growth

,Days,Growth
0,1,0.017014
1,2,0.019670
2,3,0.011578
3,4,0.013020
4,5,0.008295
5,6,0.006900
6,7,0.006871
7,8,0.006941
8,9,0.011545
9,10,0.007391


In [76]:


# Find the day with the highest 75% quantile growth
optimal_day = quantile_75_growth.loc[quantile_75_growth['Growth'].idxmax()]

print(f"Optimal number of days X: {optimal_day['Days']}, with 75% quantile growth: {optimal_day['Growth']}")

Optimal number of days X: 28.0, with 75% quantile growth: 0.03969330546173397


### Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks 
for 10 full years of data (2014-01-01 to 2023-12-31):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`
<br/>

Now let's add the top 12-22 stocks (as of end-April 2024):
<br/>

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS.

Calculate  `growth_7d` for every stock and every day.
Get the average daily `growth_7d` for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:
| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days),
and convert it to a percentage (closest INTEGER value). For example, 
if you find that 1700 out of 2595 days meet this condition, 
it means that 1700/2595 = 0.655, or approximately 66% of days, 
the LARGE stocks were growing faster than the LARGEST ones. 
This suggests that you should consider extending your dataset with more stocks to seek higher growth.

---

In [77]:
import yfinance as yf
import pandas as pd
import numpy as np

# 定義股票列表
largest_stocks = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO', 'V', 'JPM',
                  'NVO', 'MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE', 'IDEXY', 'CDI.PA',
                  'RELIANCE.NS', 'TCS.NS', 'HDB', 'BHARTIARTL.NS', 'IBN', 'SBIN.NS', 'LICI.NS', 'INFY', 'ITC.NS', 'HINDUNILVR.NS', 'LT.NS']
large_stocks = ['TSLA', 'WMT', 'XOM', 'UNH', 'MA', 'PG', 'JNJ', 'MRK', 'HD', 'COST', 'ORCL',
                'PRX.AS', 'CDI.PA', 'AIR.PA', 'SU.PA', 'ETN', 'SNY', 'BUD', 'DTE.DE', 'ALV.DE', 'MDT', 'AI.PA', 'EL.PA',
                'BAJFINANCE.NS', 'MARUTI.NS', 'HCLTECH.NS', 'TATAMOTORS.NS', 'SUNPHARMA.NS', 'ONGC.NS', 'ADANIENT.NS', 'NTPC.NS', 'KOTAKBANK.NS', 'TITAN.NS']

# 下載數據
def download_data(stocks):
    all_data = {}
    for stock in stocks:
        all_data[stock] = yf.download(stock, start='2014-01-01', end='2023-12-31')['Adj Close']
    return pd.DataFrame(all_data)

# 計算成長率
def calculate_growth(data):
    return data.pct_change(7)  # 7天的百分比變化

# 載入數據
largest_data = download_data(largest_stocks)
large_data = download_data(large_stocks)

# 計算成長率
growth_largest = calculate_growth(largest_data)
growth_large = calculate_growth(large_data)

# 計算每日平均成長率
daily_growth_largest = growth_largest.mean(axis=1)
daily_growth_large = growth_large.mean(axis=1)

# 計算大型股票組超過最大股票組的天數
outperform_days = (daily_growth_large > daily_growth_largest).sum()

# 計算百分比
total_days = len(daily_growth_large.dropna())  # 去除 NaN 值
percentage = round((outperform_days / total_days) * 100,2)

print(f"Percentage of days when LARGE stocks outperformed LARGEST stocks: {percentage}%")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Percentage of days when LARGE stocks outperformed LARGEST stocks: 46.99%



/var/folders/n3/qnzrd2254zxfjlr0q6_j679m0000gn/T/ipykernel_20404/3296506422.py:22: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return data.pct_change(7)  # 7天的百分比變化


### Question 4: Trying Another Technical Indicators strategy

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**


First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31).
If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years?
One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit.
You need to sum these results over all trades (460 times in 10 years).

Additional:
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
  * are you still profitable on those trades?

In [78]:
# import pyarrow.parquet as pq
# table = pq.read_table('stocks_df_combined_trunc_2014_2023.parquet.brotli')
# df = table.to_pandas()
# df.to_csv(r'./stocks_df_combined_trunc_2014_2023.csv')

In [83]:

import numpy as np

def calculate_cci(data, n=20):
    data = data.copy()
    # Convert columns to numeric, ignoring errors
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    # data = data[(data.index >= '2014-01-01') & (data.index <= '2023-12-31')]

    # Proceed with the CCI calculation as above
    data['TP'] = (data['High'] + data['Low'] + data['Close']) / 3
    data['MA'] = data['TP'].rolling(window=n).mean()
    data['MD'] = (data['TP'] - data['MA']).abs().rolling(window=n).mean()


    data['CCI'] = (data['TP'] - data['MA']).div(0.015 * data['MD'])
    data.fillna(0,inplace=True)
    return data


In [84]:
import yfinance as yf
import pandas as pd
import numpy as np
trades = []
def simulate_trades(cci_df):
    # Check if 'Date' is a column and convert it to datetime and set as index
    if 'Date' in cci_df.columns:
        cci_df['Date'] = pd.to_datetime(cci_df['Date'])
        cci_df.set_index('Date', inplace=True)
    elif cci_df.index.name != 'Date' or not pd.api.types.is_datetime64_any_dtype(cci_df.index):
        # If 'Date' is not the index or the index is not datetime, raise an error
        raise ValueError("DataFrame index must be 'Date' of datetime type")
    
    # Rename column Adj Close_x to Adj Close if necessary
    if 'Adj Close_x' in cci_df.columns:
        cci_df.rename(columns={'Adj Close_x': 'Adj Close'}, inplace=True)
    
    # Iterate over the DataFrame rows
    for index, row in cci_df.iterrows():
        if index.weekday() == 4 and row['CCI'] > 200:  # Only consider trades on Fridays
            buy_price = row['Adj Close']
            # Calculate sell_date as 5 trading days later
            # use the following 5 rows to calculate the sell_date
            sell_date = cci_df.loc[index:index + pd.Timedelta(days=5)].index[-1]
            if sell_date in cci_df.index:
                sell_price = cci_df.loc[sell_date, 'Adj Close']
                profit = (sell_price - buy_price) * 1000 / buy_price
                trades.append(profit)
    return trades


In [85]:
df = pd.read_csv(r'./stocks_df_combined_trunc_2014_2023.csv')
df = df[['Ticker','Date','High','Low','Close','Adj Close_x']]
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
# tickers = ['MSFT', 'AAPL', 'GOOG']  # test
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']
tickers = US_STOCKS + EU_STOCKS + INDIA_STOCKS

for ticker in tickers:
    data = df[df['Ticker'] == f'{ticker}']
    cci_df = calculate_cci(data, n=14)
    simulate_trades(cci_df)
print(sum(trades))

815.0775047679762


In [86]:
print(len(trades))

234


---
### [EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?

---
## Submitting the solutions

Form for submitting: https://courses.datatalks.club/sma-zoomcamp-2024/homework/hw02

Sorry, I don't have enough time or experience to complete the question. Perhaps I'll consider a different strategy for trading large companies.